File: prices.ipynb\
Author: alexkobz\
Date: 08-10-2024\
Task: DSRFU-298

#### Шаг 0. Импорт и получение токена

In [1]:
import requests
import pandas as pd
import numpy as np
import nest_asyncio; nest_asyncio.apply()
from datetime import datetime as dt
from math import ceil
from src.utils.get_date import last_day_month, last_day_month_str, last_work_date_month, last_work_date_month_str

# последний день месяца
REPORT_DATE = last_day_month
REPORT_DATE_STR = last_day_month_str
# увеличиваем количество столбцов и строк для просмотра
pd.set_option('display.max_rows', 200, 'display.max_columns', 200)
REPORT_DATE

Unexpected Http Driver Exception


Error HTTPConnectionPool(host='host.docker.internal', port=8124): Max retries exceeded with url: /? (Caused by NameResolutionError("<urllib3.connection.HTTPConnection object at 0x141906b00>: Failed to resolve 'host.docker.internal' ([Errno 8] nodename nor servname provided, or not known)")) executing HTTP request attempt 1 (http://host.docker.internal:8124)


datetime.date(2025, 9, 30)

In [2]:
from src.sources.rudata.RuDataMethod import Account, FintoolReferenceData, EndOfDay, AccruedInterestOnDate, FloaterData, FloatersOnPeriod, RUPriceHistory, ExchangeTree, MoexSecurities, CurrencyRate, HistoryStockBonds, HistoryStockShares, HistoryStockNdm, HistoryStockCcp
Acc = Account()

#### Шаг 1. RuData

##### Шаг 1.0. RuData stage

In [3]:
FintoolReferenceDataDF = FintoolReferenceData().df
FintoolReferenceDataDF.head()

,facevalue_m,faceftname_m,sumissuevol_m,summarketvol_m,numcoupons_m,nrdcode,regcode_m,regdate_m,regdistdate_m,begdistdate_m,enddistdate_m,endmtydate_m,daysall_m,fullname_nrd,rpregorg_nrd,rpstate_nrd,issuenumber,currentfacevalue_nrd,isqualified_nrd,bondseries,islombardcbr_nrd,numcoupons_nrd,facevalue_nrd,faceftname_nrd,summarketvol_nrd,regcode_nrd,regdate_nrd,regorg_nrd,begdistdate_nrd,daysall_nrd,enddistdate_nrd,endmtydate_nrd,regdistdate_nrd,sumissuevol_nrd,drqty,isincodebase_nrd,nrdcompcode,shqty,summarketvolorig_nrd,issuername_nrd,planmtydate_nrd,firstcoupondate_nrd,depositary_nrd,registrar_nrd,placeprice_nrd,micexlist_nrd,minlot_nrd,formarketbonds_nrd,registraracctype_nrd,registraracctypedate_nrd,facialacc_nrd,acc_open_date_nrd,fraction_nrd,specialized_depo_nrd,cmp_code_nsd_issuer_nrd,cmp_code_nsd_managing_nrd,ratetypenamerus_nrd,ratetypenameeng_nrd,coupontypename_nrd,base_month,base_year,basis_nrd,sectypenamerus_nrd,sectypenameeng_nrd,secformnamerus_nrd,secformnameeng_nrd,sectypenamebr_nrd,secstaterus_nrd,secstateeng_nrd,placementtype_nrd,share_category_name_nrd,share_category_name_en_nrd,dr_category_name_nrd,dr_category_name_en_nrd,company_state_name_nrd,rate_type_id,cp_type_id,coupon_period_base_id,sec_type_id,sec_form_id,sec_state_id,placement_type_id,share_category_id,dr_category_mn,company_state_mn,sector4212u,sna2008,fullname_en_nrd,seniorityname,fintoolid,moex_code,moex_name,fullname,isin144a,isincode,nickname,progfintoolid,regcode,regdate,regorg,amortisedmty,basis,bondstructuralpar,cfi,coupontype,fintooltype,guaranteeamount,guaranteetype,haveoffer,isconvertible,isguaranteed,maturitygroup,securitization,accruedintcalctype,country,couponperyear,faceftname,facevalue,guarantval,havedefault,numcoupons,status,sumissueval,sumissuevol,summarketval,summarketvol,borrowername,borrowerokpo,borrowersector,borroweruid,borrowerinn,borrowercountry,borrowerregionname,issuername,issuerokpo,issuersector,issueruid,issuerinn,issuercountry,issuerregionname,assetssector,numguarantors,begdistdate,begmtydate,daysall,enddistdate,endmtydate,privatedist,regdistdate,securitytype,securitykind,nominaltype,update_time,isregion,nsd_service,raterevisionsperyear,floatratename,issubordinated,note,listing_level,havecovenant,haverepayment,haveindexedfv,numdecimals,couponperiod,ismatched,firstcoupondate,couponrate,iscomplicated,available_for_unqualified,sec_type_br_code,original_country,qualified_test_id,qualified_test_id_tm,base_asset_fintoolid,isqualified,listing_level_spb,have_mm_moex,have_mm_spb,beg_book_date,seniority,tradesite,current_coupontype,profit_type,counter,rn
0,2.0,2.0,2.0,2.0,2.0,None,0.0,0.0,2.0,2.0,2.0,2.0,2.0,None,None,None,None,NaN,NaN,None,NaN,NaN,NaN,None,NaN,None,None,None,None,NaN,None,None,None,NaN,NaN,None,None,NaN,NaN,None,None,None,None,None,NaN,None,NaN,NaN,None,None,None,None,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,520,S2,None,Старшие необеспеченные,226059,None,None,"Соединённые Штаты Америки, 5-летние ноты с пог...",None,US91282CGH88,США-2-280131-об,NaN,None,None,None,0.0,act/act,None,None,Постоянный,Облигация,None,Необеспеченные,0.0,0.0,0.0,Среднесрочные,None,1.0,US,2.0,USD,100.00,NaN,0.0,10.0,В обращении,4.400000e+10,4.400000e+08,4.300001e+10,4.300001e+08,Правительство США,None,Государственные,95111.0,None,US,None,Минфин США,None,Государственные,95112.0,None,US,None,None,0.0,2023-01-31T00:00:00,None,1826.0,2023-01-31T00:00:00,2028-01-31T00:00:00,0.0,2023-01-31T00:00:00,Гос,Классические,Постоянный,2025-08-15T10:20:36,NaN,Нет информации,2.0,None,NaN,None,NaN,False,False,False,2.0,184.0,True,2023-01-31T00:00:00,3.5,False,0,OTHER,US,1000,1000,NaN,False,NaN,0,0,None,416.0,NaN,Фиксированный,Постоянный,106489,90113
1,2.0,2.0,2.0,2.0,2.0,None,0.0,0.0,2.0,2.0,2.0,2.0,2.0,None,None,None,None,NaN,NaN,None,NaN,NaN,NaN,None,NaN,None,None,None,None,NaN,None,None,None,NaN,NaN,None,None,NaN,NaN,None,None,None,None,None,Na

In [4]:
EndOfDayDF = EndOfDay().df
EndOfDayDF.head()

,ask,avge_prce,bid,boardid,boardname,close,convexity,deal_acc,duration,exch,high,isin,last,last_time,last_yield,low,mat_date,mprice,name,open,pvbp,seccode,secname,spread,val,val_usd,vol,vol_lots,yield_agg,yield_ask,yield_average_o12m,yield_average_o2w,yield_bid,yield_swa,currency,fintoolId,id_iss,id_trade_site,add_date,update_date
0,54.34,54.510000,54.32,TQBR,T+ Акции и ДР,54.68,NaN,27247.0,NaN,MICEX,55.10,RU0009062285,54.34,2025-09-30T23:49:53,NaN,54.07,None,54.49,AFLT.MICEX,54.96,NaN,AFLT,"Аэрофлот - российские авиалинии ОАО, АО - ФБ...",0.02,963205621.0,11623091.84,17668670.0,1766867.0,NaN,NaN,NaN,NaN,NaN,NaN,RUB,4200,35154,170,2025-09-30T23:55:00,2025-09-30T23:55:00
1,9.09,9.120000,9.05,TQBR,T+ Акции и ДР,9.27,NaN,66.0,NaN,MICEX,9.27,RU0009100176,9.09,2025-09-30T23:40:46,NaN,9.02,None,9.25,BISVP.MICEX,9.27,NaN,BISVP,"Башинформсвязь ОАО, АП - ФБ ММВБ",0.04,109400.0,1320.13,12000.0,120.0,NaN,NaN,NaN,NaN,NaN,NaN,RUB,4215,775000,170,2025-09-30T23:55:06,2025-09-30T23:55:06
2,1602.00,1611.515152,1594.00,TQBR,T+ Акции и ДР,1620.00,NaN,125.0,NaN,MICEX,1642.00,RU0009288658,1596.00,2025-09-30T22:54:04.0001,NaN,1576.00,None,1600.00,BRZL.MICEX,1624.00,NaN,BRZL,"Бурятзолото ОАО, АО - ФБ ММВБ",8.00,1063600.0,12834.56,660.0,660.0,NaN,NaN,NaN,NaN,NaN,NaN,RUB,4217,852973,170,2025-09-30T23:55:08,2025-09-30T23:55:08
3,82.80,83.900000,82.20,TQBR,T+ Акции и ДР,80.00,NaN,408.0,NaN,MICEX,88.80,RU0009082291,82.80,2025-09-30T23:47:29,NaN,79.00,None,80.80,CHGZ.MICEX,80.00,NaN,CHGZ,"Черногорнефть ОАО, АО - ФБ ММВБ",0.60,3575104.0,43141.11,42600.0,4260.0,NaN,NaN,NaN,NaN,NaN,NaN,RUB,4223,853518,170,2025-09-30T23:55:08,2025-09-30T23:55:08
4,944.00,951.700000,943.20,TQBR,T+ Акции и ДР,960.60,NaN,26087.0,NaN,MICEX,965.00,RU0009046510,943.20,2025-09-30T23:49:30,NaN,941.00,None,951.60,CHMF.MICEX,960.60,NaN,CHMF,"Северсталь ОАО, АО - ФБ ММВБ T+2",0.80,905113061.0,10922083.51,951055.0,951055.0,NaN,NaN,NaN,NaN,NaN,NaN,RUB,4226,1069768,170,2025-09-30T23:55:06,2025-09-30T23:55:06


In [5]:
AccruedInterestOnDateDF = AccruedInterestOnDate().df
AccruedInterestOnDateDF.head()

,fintoolId,shiftDays,shiftTypeSource,typeCurrency,date,accruedInterest,accruedInterestPct,currentFaceValue,currentFaceValuePct,calculationType,error,faceValueCurrency,accruedInterestCurrency,faceValueLastKnownDate
0,4000,0,Calendar,None,2025-09-30T00:00:00,NaN,NaN,1000.0,100.0,Typical,None,RUB,RUB,2025-09-30T00:00:00
1,4001,0,Calendar,None,2025-09-30T00:00:00,NaN,NaN,1000.0,100.0,Typical,None,RUB,RUB,2025-09-30T00:00:00
2,4002,0,Calendar,None,2025-09-30T00:00:00,NaN,NaN,1000.0,100.0,Typical,None,RUB,RUB,2025-09-30T00:00:00
3,4003,0,Calendar,None,2025-09-30T00:00:00,NaN,NaN,1000.0,100.0,Typical,None,RUB,RUB,2025-09-30T00:00:00
4,4004,0,Calendar,None,2025-09-30T00:00:00,NaN,NaN,1000.0,100.0,Typical,None,RUB,RUB,2025-09-30T00:00:00


In [6]:
FloaterDataDF = FloaterData().df
FloaterDataDF.head()

,fintoolId,beg_period,end_period,start_point,model_type,date_pub_n,date_pub_type,date_fix_n,date_fix_type,bases
0,11193,2000-12-01T00:00:00,2050-12-01T00:00:00,E,single,NaN,None,10.0,W,"[{""base_no"": 1, ""calc_type"": ""FIX"", ""base_fint..."
1,30316,2025-09-18T00:00:00,2025-12-18T00:00:00,B,single,NaN,None,2.0,W,"[{""base_no"": 5, ""calc_type"": ""FIX"", ""base_fint..."
2,36325,2009-08-27T00:00:00,2028-09-15T00:00:00,B,single,NaN,None,1.0,W,"[{""base_no"": 1, ""calc_type"": ""FIX"", ""base_fint..."
3,57394,2011-11-21T00:00:00,2028-10-30T00:00:00,B,single,NaN,None,5.0,W,"[{""base_no"": 2, ""calc_type"": ""PER"", ""base_fint..."
4,59702,2013-12-11T00:00:00,2028-11-22T00:00:00,B,single,NaN,None,5.0,C,"[{""base_no"": 2, ""calc_type"": ""FIX"", ""base_fint..."


In [7]:
FloatersOnPeriodDF = FloatersOnPeriod().df
FloatersOnPeriodDF.head()

,fintoolId,date,accruedInterest,accruedInterestPct,currentFaceValue,currentFaceValuePct,faceValueCurrency,addDate,counter
0,164245,2025-09-30T00:00:00,2.78,0.28,1000.0,100.0,RUB,2025-09-29T06:04:23,1
1,165152,2025-09-30T00:00:00,2.78,0.28,1000.0,100.0,RUB,2025-09-29T06:04:29,1
2,170151,2025-09-30T00:00:00,33.75,3.37,1000.0,100.0,RUB,2025-09-29T06:04:44,2
3,170152,2025-09-30T00:00:00,2.78,0.28,1000.0,100.0,RUB,2025-09-29T06:04:50,2
4,170785,2025-09-30T00:00:00,12.82,1.28,1000.0,100.0,RUB,2025-09-29T06:05:01,4


In [8]:
RUPriceHistoryDF = RUPriceHistory().df
RUPriceHistoryDF.head()

,nickname,fintoolid,fininstid,isincode,regcode,nrdcode,calcdate,val_date,current_fv,faceftname,date_agg,avge_prce,yield_agg,z_wa_mb,durmod_agg,i_wa_mb,method_no,vp_pct,vp_pc,vp_pc_l,vp_pc_h,vp_m1,y_agg_m1,deal_250d,q_m1,r_m1,vp_m2,y_agg_m2,i_vp_m2,sigma_m2,index,beta1_m2,beta0_m2,alpha_m2,gamma_m2,sigma_nu,rho_m2,r_m2,vp_m3,y_agg_m3,dur_agg_m3,z_vp_m3,r_m3,factor1,beta1_m3,factor2,beta2_m3,level_pd,coef_b,risk_b,beta3_m3,is_ontherun,beta4_m3,hhi,beta5_m3,ind_liq,beta6_m3,ind_unliq,beta7_m3,sector,phi_sector,version,valuation_currency,security_type,source,wa_duration,duration_mod,lbound1,ubound1,lbound2,ubound2,lbound3,ubound3,acc_int,wap_ext_zspread,mp3_ext_zspread,id,no_history_found,not_valid_code,calcDateTime,counter,rn
0,Россия-2030-11т,6043,7080,XS0114288789,SK-0-CM-128,XS0114288789,2025-09-01T00:00:00,2025-09-01T00:00:00,0.05,USD,2030-03-31T00:00:00,0.0,0.0,0.0,None,0.0,1.0,107.451809,0.053726,105.825836,109.077782,107.451809,4.097067,0.0,0.0,0.0,107.451809,4.097067,0.0,0.0,None,0.0,0.0,0.0,0.0,0.0,0.0,0.0,107.451809,4.097067,0.0,0.3399,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Государственные,0.0,2,USD,euro_bond,3,0.0,2.034537,105.825836,109.077782,105.825836,109.077782,105.825836,109.077782,0.001573,0.0,0.0,6043,None,None,None,45922,1
1,Россия-2030-11т,6043,7080,XS0114288789,SK-0-CM-128,XS0114288789,2025-09-02T00:00:00,2025-09-02T00:00:00,0.05,USD,2030-03-31T00:00:00,0.0,0.0,0.0,None,0.0,1.0,107.394802,0.053697,105.845938,108.943667,107.394802,4.118709,0.0,0.0,0.0,107.394802,4.118709,0.0,0.0,None,0.0,0.0,0.0,0.0,0.0,0.0,0.0,107.394802,4.118709,0.0,0.3640,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Государственные,0.0,2,USD,euro_bond,3,0.0,2.031078,105.845938,108.943667,105.845938,108.943667,105.845938,108.943667,0.001583,0.0,0.0,6043,None,None,None,45922,2
2,Россия-2030-11т,6043,7080,XS0114288789,SK-0-CM-128,XS0114288789,2025-09-03T00:00:00,2025-09-03T00:00:00,0.05,USD,2030-03-31T00:00:00,0.0,0.0,0.0,None,0.0,1.0,107.355400,0.053678,106.166814,108.543986,107.355400,4.132569,0.0,0.0,0.0,107.355400,4.132569,0.0,0.0,None,0.0,0.0,0.0,0.0,0.0,0.0,0.0,107.355400,4.132569,0.0,0.3753,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Государственные,0.0,2,USD,euro_bond,3,0.0,2.027918,106.166814,108.543986,106.166814,108.543986,106.166814,108.543986,0.001594,0.0,0.0,6043,None,None,None,45922,3
3,Россия-2030-11т,6043,7080,XS0114288789,SK-0-CM-128,XS0114288789,2025-09-04T00:00:00,2025-09-04T00:00:00,0.05,USD,2030-03-31T00:00:00,0.0,0.0,0.0,None,0.0,1.0,106.876019,0.053438,105.421696,108.330343,106.876019,4.343805,0.0,0.0,0.0,106.876019,4.343805,0.0,0.0,None,0.0,0.0,0.0,0.0,0.0,0.0,0.0,106.876019,4.343805,0.0,0.4306,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Государственные,0.0,2,USD,euro_bond,3,0.0,2.017252,105.421696,108.330343,105.421696,108.330343,105.421696,108.330343,0.001604,0.0,0.0,6043,None,None,None,45922,4
4,Россия-2030-11т,6043,7080,XS0114288789,SK-0-CM-128,XS0114288789,2025-09-05T00:00:00,2025-09-05T00:00:00,0.05,USD,2030-03-31T00:00:00,0.0,0.0,0.0,None,0.0,1.0,105.762540,0.052881,103.802379,107.722701,105.762540,4.847146,0.0,0.0,0.0,105.762540,4.847146,0.0,0.0,None,0.0,0.0,0.0,0.0,0.0,0.0,0.0,105.762540,4.847146,0.0,0.3631,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Государственные,0.0,2,USD,euro_bond,3,0.0,1.995665,103.802379,107.722701,103.802379,107.722701,103.802379,107.722701,0.001615,0.0,0.0,6043,None,None,None,45922,5


In [9]:
ExchangeTree = ExchangeTree().df
ExchangeTree.head()

,id,id_parent,shortname_rus,shortname_eng,fullname_rus,fullname_eng,sort_order,update_date,counter,rn
0,1,0,РТС,RTS,Московская Биржа - РТС,RTS,12,2020-12-28T17:03:09,242,17
1,2,1,РТС,RTS,РТС индексы,RTS indexes,2,2020-12-28T17:03:09,242,4
2,8,1,ФОРТС,FORTS,FORTS,FORTS,8,2020-12-28T17:03:09,242,11
3,9,0,"ПАО ""СПБ""",PJSC «SPB»,"ПАО ""СПБ""",PJSC «SPB»,20,2020-12-28T17:03:09,242,21
4,11,0,МБ,MX,Московская Биржа,Moscow Exchange,11,2020-12-28T17:03:09,242,16


In [10]:
MoexSecuritiesDF = MoexSecurities().df
MoexSecuritiesDF.head()

,id_iss,secid,boardid,board_title,board_group,board_group_title,market,engine,short_name,isin,cfi_ifx,cfi_name,lot_size,expiration_date,currency,emitent_code,emitent_name,market_sector,min_price_increment,fintoolid,reg_code,update_date,counter,rn
0,3140664,SLDB-RM,FSEQ,РПС: Ин.Акции - адрес.,stock_foreign_ndm,РПС: Ин.Акции - адрес.,foreignndm,stock,None,US83422E2046,ESXXXX,Акции / обыкновенные,1,None,None,None,None,None,1.000000,172138.0,None,2022-11-01T10:06:35.26734,227967,111451
1,3949491,SLDB-RM,MPTR,ОТС: Акции РПС с ЦК - адресные,otc_ndm_ccp_shares_rub,ОТС: Акции РПС с ЦК - адресные,sharesndm,otc,None,US83422E2046,ESXXXX,Акции / обыкновенные,1,None,None,None,None,None,1.000000,172138.0,None,2025-07-28T06:36:44.2759,227967,221265
2,3949555,SLDB-RM,MTQR,ОТС: Акции Т+ - безадресные,otc_shares_tplus,ОТС: Акции Т+ - безадресные,shares,otc,None,US83422E2046,ESXXXX,Акции / обыкновенные,1,None,None,None,None,None,1.000000,172138.0,None,2025-07-28T06:36:50.275965,227967,221329
3,1398554,SLEN,PSEQ,РПС: Акции - адрес.,stock_ndm,РПС - адрес.,ndm,stock,Сахэнер-АО,RU0009280465,ESXXXX,Акции / обыкновенные,1,None,RUB,SLEN,Сахалинэнерго,Электроэнергетика,0.005000,6581.0,1-03-00272-A,2025-07-03T13:20:12.98197,227967,24676
4,1398555,SLEN,RPEU,РЕПО в ин. валюте (USD) - адрес.,stock_repo_interdealer_usd,Междилерское РЕПО (USD) - адрес.,repo,stock,Сахэнер-АО,RU0009280465,ESXXXX,Акции / обыкновенные,1,None,RUB,SLEN,Сахалинэнерго,Электроэнергетика,0.000001,6581.0,1-03-00272-A,2025-07-03T13:20:12.98197,227967,24677


In [11]:
CurrencyRateDF = CurrencyRate().df
CurrencyRateDF.head()

,rate,endOfDayRate,error,currency
0,NaN,NaN,None,ALL
1,0.639655,0.638054,None,DZD
2,NaN,NaN,None,ARS
3,54.452300,54.538100,None,AUD
4,NaN,NaN,None,BSD


##### Шаг 1.1. RuData MOEX stage

In [12]:
HistoryStockBondsDF = HistoryStockBonds().df
HistoryStockBondsDF.head()

,boardname,regnumber,isin,matdate,listname,facevalue,currencyid,volume,value,numtrades,prev,prevlegalcloseprice,openperiod,legalopenprice,legalcloseprice,closeperiod,waprice,trendclose,trendclspr,trendwap,trendwappr,openval,closeval,highbid,lowoffer,bid,offer,yieldatwap,yieldclose,accint,duration,marketprice,marketprice2,admittedquote,issuesize,mpvaltrd,mp2valtrd,admittedvalue,marketprice3,marketprice3tradesvalue,type,lastprice,iricpiclose,beiclose,couponpercent,couponvalue,buybackdate,lasttradedate,cbrclose,yieldtooffer,yieldlastcoupon,offerdate,faceunit,engine,market,boardid,decimals,high,low,open,close,secid,shortname,tradedate,tradingsession,counter
0,Т+: Облигации - безадрес.,MF-LB-USD-0331,BYM000001818,2027-06-29T00:00:00,3,1000.0,SUR,130,10357819.47,12,98.8000,99.1680,NaN,None,98.9362,None,99.1833,0.4000,0.40,0.1171,0.12,79447.95,79688.95,99.2000,98.6724,98.6852,99.1817,8.27,8.26,1074.0225,NaN,None,99.1833,None,240419,0.0,10357819.47,None,99.1833,10357819.47,corporate_bond,99.2000,NaN,NaN,7.63,38.23,None,2025-09-01T00:00:00,None,NaN,NaN,None,USD,stock,bonds,TQCB,4,99.2000,98.6724,98.9000,99.2000,BYM000001818,РесБел 331,2025-09-01T00:00:00,3,NaN
1,Т+: Облигации - безадрес.,MF-LB-USD-0331,BYM000001818,2027-06-29T00:00:00,3,1000.0,SUR,50,3977873.36,25,99.2000,98.9362,NaN,None,99.0041,None,98.9200,-0.6000,-0.60,-0.2633,-0.27,159243.68,237900.40,99.0111,98.5000,98.6000,98.9977,8.44,8.64,1092.0873,NaN,None,98.9899,None,240419,0.0,3264162.27,None,98.9899,3264162.27,corporate_bond,98.6000,NaN,NaN,7.63,38.23,None,2025-09-02T00:00:00,None,NaN,NaN,None,USD,stock,bonds,TQCB,4,99.0105,98.6000,99.0000,98.6000,BYM000001818,РесБел 331,2025-09-02T00:00:00,3,NaN
2,Т+: Облигации - безадрес.,MF-LB-USD-0331,BYM000001818,2027-06-29T00:00:00,3,1000.0,SUR,426,34109935.56,41,98.6000,99.0041,NaN,None,98.6035,None,99.3493,0.5769,0.59,0.4293,0.43,79699.94,239793.98,99.5000,98.8898,98.7001,99.4054,8.17,8.28,1111.2132,NaN,None,99.3549,None,240419,0.0,32830674.74,None,99.3549,32830674.74,corporate_bond,99.1769,NaN,NaN,7.63,38.23,None,2025-09-03T00:00:00,None,NaN,NaN,None,USD,stock,bonds,TQCB,4,99.5000,98.6003,98.8898,99.1769,BYM000001818,РесБел 331,2025-09-03T00:00:00,3,NaN
3,Т+: Облигации - безадрес.,MF-LB-USD-0331,BYM000001818,2027-06-29T00:00:00,3,1000.0,SUR,507,40770489.85,39,99.1769,98.6035,NaN,None,99.2005,None,99.4636,0.4020,0.41,0.1143,0.12,80185.84,80508.35,99.5789,99.1800,99.2013,99.5789,8.10,8.02,1131.6063,NaN,None,99.4632,None,240419,0.0,40609473.24,None,99.4632,40609473.24,corporate_bond,99.5789,NaN,NaN,7.63,38.23,None,2025-09-04T00:00:00,None,NaN,NaN,None,USD,stock,bonds,TQCB,4,99.5789,99.1800,99.1800,99.5789,BYM000001818,РесБел 331,2025-09-04T00:00:00,3,NaN
4,Т+: Облигации - безадрес.,MF-LB-USD-0331,BYM000001818,2027-06-29T00:00:00,3,1000.0,SUR,60,4849121.30,21,99.5789,99.2005,NaN,None,99.3444,None,99.4108,0.2210,0.22,-0.0528,-0.05,242830.70,81135.02,99.7497,99.5643,99.2520,99.8000,8.13,7.89,1154.8728,NaN,None,99.4042,None,240419,0.0,4767986.28,None,99.4042,4767986.28,corporate_bond,99.7999,NaN,NaN,7.63,38.23,None,2025-09-05T00:00:00,None,NaN,NaN,None,USD,stock,bonds,TQCB,4,99.8000,99.2500,99.5644,99.7999,BYM000001818,РесБел 331,2025-09-05T00:00:00,3,NaN


In [13]:
HistoryStockSharesDF = HistoryStockShares().df
HistoryStockSharesDF.head()

,boardname,regnumber,isin,listname,facevalue,currencyid,prevlegalcloseprice,prev,legalopenprice,openperiod,closeperiod,openval,closeval,legalcloseprice,trendclose,trendclspr,highbid,bid,offer,lowoffer,waprice,trendwap,trendwappr,volume,marketprice,marketprice2,mp2valtrd,mpvaltrd,value,numtrades,issuesize,admittedquote,admittedvalue,monthlycapitalization,dailycapitalization,marketprice3,marketprice3tradesvalue,type,closeauctionprice,waval,lastprice,engine,market,boardid,decimals,high,low,open,close,secid,shortname,tradedate,tradingsession,counter
0,Поставка по СК (акции),1-01-08902-A,RU000A0JNAB6,2,0.1,SUR,NaN,76.20,None,NaN,None,3625.0,133400.0,NaN,-3.70,-4.86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29260,None,NaN,0.0,0.0,2121350.0,36,92645451,None,None,NaN,NaN,NaN,0.0,common_share,NaN,0.0,NaN,stock,shares,SPEQ,6,72.50,72.50,72.50,72.50,ABIO,iАРТГЕН ао,2025-09-19T00:00:00,3,NaN
1,Поставка по СК (акции),1-01-08902-A,RU000A0JNAB6,2,0.1,SUR,NaN,76.20,None,NaN,None,3625.0,133400.0,NaN,-3.70,-4.86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29260,None,NaN,0.0,0.0,2121350.0,36,92645451,None,None,NaN,NaN,NaN,0.0,common_share,NaN,0.0,NaN,stock,shares,SPEQ,6,72.50,72.50,72.50,72.50,ABIO,iАРТГЕН ао,2025-09-19T00:00:00,1,NaN
2,Т+: Акции и ДР - безадрес.,1-01-08902-A,RU000A0JNAB6,2,0.1,SUR,76.68,76.84,None,NaN,None,2305.2,775.0,77.40,0.66,0.86,79.96,77.22,77.42,76.20,77.96,0.96,1.25,229930,None,78.12,13009420.8,0.0,17923099.0,1445,92645451,None,None,7.131847e+09,7.237463e+09,78.12,13009420.8,common_share,77.40,0.0,77.50,stock,shares,TQBR,2,78.52,76.72,76.84,77.50,ABIO,iАРТГЕН ао,2025-09-01T00:00:00,3,NaN
3,Т+: Акции и ДР - безадрес.,1-01-08902-A,RU000A0JNAB6,2,0.1,SUR,77.40,77.50,None,NaN,None,775.2,1538.8,77.02,-0.56,-0.72,79.36,76.92,77.06,75.92,77.18,-0.78,-1.00,94910,None,77.26,4983911.4,0.0,7325182.6,1122,92645451,None,None,7.131847e+09,7.157788e+09,77.26,4983911.4,common_share,77.02,0.0,76.94,stock,shares,TQBR,2,77.74,76.68,77.52,76.94,ABIO,iАРТГЕН ао,2025-09-02T00:00:00,3,NaN
4,Т+: Акции и ДР - безадрес.,1-01-08902-A,RU000A0JNAB6,2,0.1,SUR,77.02,76.94,None,NaN,None,772.6,773.2,76.90,0.38,0.49,79.38,77.14,77.34,75.62,77.08,-0.10,-0.13,56260,None,77.00,2604177.2,0.0,4336377.0,500,92645451,None,None,7.131847e+09,7.133700e+09,77.00,2604177.2,common_share,76.90,0.0,77.32,stock,shares,TQBR,2,77.48,76.64,77.26,77.32,ABIO,iАРТГЕН ао,2025-09-03T00:00:00,3,NaN


In [14]:
HistoryStockNdmDF = HistoryStockNdm().df
HistoryStockNdmDF.head()

,boardname,closeval,currencyid,facevalue,isin,issuesize,listname,matdate,numtrades,openval,prev,regnumber,trendclose,trendclspr,trendwap,trendwappr,type,value,volume,waprice,yieldatwap,yieldclose,lastprice,iricpiclose,beiclose,couponpercent,couponvalue,buybackdate,lasttradedate,cbrclose,yieldtooffer,yieldlastcoupon,offerdate,faceunit,engine,market,boardid,decimals,high,low,open,close,secid,shortname,tradedate,tradingsession,counter
0,РПС: Акции - адрес.,1398.7,SUR,0.09,RU000A0DQZE3,9650000000,1,None,1,1398.7,15.516,1-05-01669-A,-1.529,NaN,-0.111,-0.79,common_share,1398.7,100,13.960,NaN,NaN,None,NaN,NaN,NaN,NaN,None,None,None,NaN,NaN,None,None,stock,ndm,PSEQ,3,13.987,13.987,13.987,13.987,AFKS,Система ао,2025-09-26T00:00:00,3,NaN
1,РПС с ЦК: Акции и ДР - адрес.,58922316.8,SUR,0.09,RU000A0DQZE3,9650000000,1,None,1,58922316.8,16.400,1-05-01669-A,-0.444,NaN,-0.229,-1.41,common_share,58922316.8,3692800,16.030,NaN,NaN,None,NaN,NaN,NaN,NaN,None,None,None,NaN,NaN,None,None,stock,ndm,PTEQ,3,15.956,15.956,15.956,15.956,AFKS,Система ао,2025-09-02T00:00:00,3,NaN
2,РПС с ЦК: Акции и ДР - адрес.,37790288.0,SUR,0.09,RU000A0DQZE3,9650000000,1,None,1,37790288.0,15.956,1-05-01669-A,0.524,NaN,-0.020,-0.12,common_share,37790288.0,2293100,16.486,NaN,NaN,None,NaN,NaN,NaN,NaN,None,None,None,NaN,NaN,None,None,stock,ndm,PTEQ,3,16.480,16.480,16.480,16.480,AFKS,Система ао,2025-09-05T00:00:00,3,NaN
3,РПС с ЦК: Акции и ДР - адрес.,18590628.4,SUR,0.09,RU000A0DQZE3,9650000000,1,None,1,18590628.4,16.480,1-05-01669-A,-0.437,NaN,-0.302,-1.85,common_share,18590628.4,1158800,16.044,NaN,NaN,None,NaN,NaN,NaN,NaN,None,None,None,NaN,NaN,None,None,stock,ndm,PTEQ,3,16.043,16.043,16.043,16.043,AFKS,Система ао,2025-09-11T00:00:00,3,NaN
4,РПС с ЦК: Акции и ДР - адрес.,29544045.3,SUR,0.09,RU000A0DQZE3,9650000000,1,None,1,29544045.3,16.043,1-05-01669-A,-0.566,NaN,-0.351,-2.19,common_share,29544045.3,1908900,15.693,NaN,NaN,None,NaN,NaN,NaN,NaN,None,None,None,NaN,NaN,None,None,stock,ndm,PTEQ,3,15.477,15.477,15.477,15.477,AFKS,Система ао,2025-09-12T00:00:00,3,NaN


In [15]:
HistoryStockCcpDF = HistoryStockCcp().df
HistoryStockCcpDF.head()

,accint,admittedquote,admittedvalue,bid,boardname,closeperiod,closeval,currencyid,duration,facevalue,highbid,isin,issuesize,legalcloseprice,legalopenprice,listname,lowoffer,marketprice,marketprice2,marketprice3,marketprice3tradesvalue,matdate,mp2valtrd,mpvaltrd,numtrades,offer,openperiod,openval,prev,prevlegalcloseprice,regnumber,trendclose,trendclspr,trendwap,trendwappr,type,value,volume,waprice,yieldatwap,yieldclose,engine,market,boardid,decimals,high,low,open,close,secid,shortname,tradedate,tradingsession,counter
0,NaN,None,None,None,РЕПО с ЦК 1 день - безадрес.,None,99990.96,SUR,None,0.1,NaN,RU000A0JNAB6,92645451,None,None,2,NaN,NaN,None,None,NaN,None,NaN,NaN,5,None,None,105342.43,17.01,None,1-01-08902-A,NaN,None,NaN,NaN,common_share,778114.67,13377,17.01,None,None,stock,ccp,EQRP,4,17.01,17.01,17.01,17.01,ABIO,iАРТГЕН ао,2025-09-01T00:00:00,3,NaN
1,NaN,None,None,None,РЕПО с ЦК 1 день - безадрес.,None,367895.82,SUR,None,0.1,NaN,RU000A0JNAB6,92645451,None,None,2,NaN,NaN,None,None,NaN,None,NaN,NaN,4,None,None,104961.97,17.01,None,1-01-08902-A,NaN,None,NaN,NaN,common_share,678080.18,11538,17.01,None,None,stock,ccp,EQRP,4,17.01,17.01,17.01,17.01,ABIO,iАРТГЕН ао,2025-09-02T00:00:00,3,NaN
2,NaN,None,None,None,РЕПО с ЦК 1 день - безадрес.,None,100027.65,SUR,None,0.1,NaN,RU000A0JNAB6,92645451,None,None,2,NaN,NaN,None,None,NaN,None,NaN,NaN,5,None,None,104999.78,17.01,None,1-01-08902-A,NaN,None,0.37,NaN,common_share,832803.28,14237,17.38,None,None,stock,ccp,EQRP,4,17.75,17.01,17.01,17.01,ABIO,iАРТГЕН ао,2025-09-03T00:00:00,3,NaN
3,NaN,None,None,None,РЕПО с ЦК 1 день - безадрес.,None,104049.73,SUR,None,0.1,NaN,RU000A0JNAB6,92645451,None,None,2,NaN,NaN,None,None,NaN,None,NaN,NaN,4,None,None,517913.09,17.01,None,1-01-08902-A,NaN,None,-0.37,NaN,common_share,829128.06,14200,17.01,None,None,stock,ccp,EQRP,4,17.01,17.01,17.01,17.01,ABIO,iАРТГЕН ао,2025-09-04T00:00:00,3,NaN
4,NaN,None,None,None,РЕПО с ЦК 1 день - безадрес.,None,639271.27,SUR,None,0.1,NaN,RU000A0JNAB6,92645451,None,None,2,NaN,NaN,None,None,NaN,None,NaN,NaN,6,None,None,112761.16,17.01,None,1-01-08902-A,0.79,None,0.29,NaN,common_share,1714377.52,29419,17.30,None,None,stock,ccp,EQRP,4,17.80,17.01,17.01,17.80,ABIO,iАРТГЕН ао,2025-09-05T00:00:00,3,NaN


##### Шаг 1.1. Discounts

In [17]:
# Prices = pd.DataFrame(
#     index=FintoolReferenceDataDF['isincode'].dropna().unique(),
#     columns=
#         ["Name", "Type", "Principal", "Currency", "Clean price pct", "Clean price", "Interest", "Price",
#         "Source", "ISIN in DB", "FX Rate", "RUB Price", "CBR discount 6D", "Discount NCC", "NCC collateral"]
# )
# Prices.index.name = 'ISIN'
# Prices["Date"] = REPORT_DATE

##### Шаг 2. RuData transform

In [16]:
Prices = FintoolReferenceDataDF[['isincode', 'nickname', 'fintooltype', 'fintoolid', 'faceftname']]\
.rename(columns={
    "isincode": "ISIN",
    "nickname": "Name",
    "fintoolid": "Fintool",
    "fintooltype": "Type",
    "faceftname": "Currency"
})
Prices = Prices[~(Prices["ISIN"].isna() | (Prices["ISIN"].isin(['None', ''])))]
InterestDF = AccruedInterestOnDateDF[['fintoolId', 'accruedInterest', 'currentFaceValue']].set_index('fintoolId')
InterestDF.update(FloatersOnPeriodDF.groupby('fintoolId').last())
Prices = Prices.merge(
    InterestDF, how="left", left_on="Fintool", right_index=True)
Prices = Prices.rename(columns={
    'accruedInterest': 'Interest',
    'currentFaceValue': 'Principal'})
Prices = Prices.merge(CurrencyRateDF[['currency', 'rate']], how='left', left_on='Currency', right_on='currency').drop(columns=['currency'])
Prices['Source'] = ''

###### Шаг 1.1.1. CBR

In [18]:
from src.sources.cbr.CBR_Soap import CBR_Soap

headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.6834.111 Safari/537.36', 
           'Accept-Encoding': 'gzip, deflate', 
           'Accept': '*/*', 
           'Connection': 'keep-alive'}
cbr_disc = CBR_Soap()\
.get_discounts(date=last_work_date_month_str)[['ISIN', 'от 1 месяца до 180 дней_Beg']]\
.rename(columns={'от 1 месяца до 180 дней_Beg': 'CBR discount 6D'})\
.groupby('ISIN')\
.last()\
.reset_index()

GET http://www.cbr.ru/DailyInfoWebServ/DailyInfo.asmx?wsdl using urllib2 3.10
GET http://www.cbr.ru/DailyInfoWebServ/DailyInfo.asmx?wsdl using urllib2 3.10
GET http://www.cbr.ru/secinfo/secinfo.asmx?wsdl using urllib2 3.10
GET http://www.cbr.ru/secinfo/secinfo.asmx?wsdl using urllib2 3.10


/Users/alexander/PycharmProjects/insurance_mine/src/sources/cbr/CBR_Soap.py:212: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  disc = pd.concat([disc, pd.DataFrame(index=[0], data=d)], axis=0)


In [19]:
Prices = Prices.merge(cbr_disc, how='left', left_on='ISIN', right_on='ISIN')
Prices['CBR discount 6D'] = Prices['CBR discount 6D'].fillna(1.0)

###### Шаг 1.1.2. nationalclearingcentre Рыночные риски

In [20]:
currMarketRatesResponse = requests.get(f'https://www.nationalclearingcentre.ru/api/v1/rates/fondMarketRates?action=xls&xls=1&lang=ru&date={last_work_date_month.strftime("%d.%m.%Y")}', headers=headers)
currMarketRates = pd.read_excel(currMarketRatesResponse.content,header=2)
currMarketRates = currMarketRates[(currMarketRates['Диапазон']==1)&(currMarketRates['Ставка риска падения цены,%']!=100)].reset_index(drop=True)
currMarketRates = currMarketRates\
.merge(MoexSecuritiesDF[['secid', 'isin']], how='left', left_on='Инструмент', right_on='secid')\
.rename(columns={'isin': 'ISIN'})\
.groupby('ISIN')\
.last()\
.reset_index()
currMarketRates = currMarketRates[currMarketRates['ISIN'].str.match('([A-Z]{2})([A-Z0-9]{9})([0-9]{1})$')]
currMarketRates = currMarketRates[
    (currMarketRates['Диапазон']==1) & (currMarketRates['Ставка риска падения цены,%'] != 100)
]
currMarketRates=currMarketRates[['ISIN', 'Ставка риска падения цены,%']]\
.rename(columns={'Ставка риска падения цены,%': 'Risk rate'})
# Формула из п.12.1 Методики НКЦ определения риск-параметров
currMarketRates['Discount NCC'] = currMarketRates['Risk rate'].apply(lambda x: min(30, ceil(x/np.sqrt(2)))/100)

/var/folders/g4/mpm4njl93p547kbhxbvqlm480000gn/T/ipykernel_19167/4206194447.py:2: FutureWarning: Passing bytes to 'read_excel' is deprecated and will be removed in a future version. To read from a byte string, wrap it in a `BytesIO` object.
  currMarketRates = pd.read_excel(currMarketRatesResponse.content,header=2)


In [21]:
Prices = Prices.merge(currMarketRates[['ISIN', 'Discount NCC']], how='left', left_on='ISIN', right_on='ISIN')
Prices['Discount NCC'] = Prices['Discount NCC'].fillna(1.0)

###### Шаг 1.1.3. nationalclearingcentre Параметры ценных бумаг

In [22]:
securInfoResponse = requests.get(f'https://www.nationalclearingcentre.ru/api/v1/rates/securInfo?action=xls&lang=ru&settleDate={last_work_date_month.strftime("%d.%m.%Y")}', headers=headers)
securInfo = pd.read_excel(securInfoResponse.content, header=2)[['ISIN ЦБ', 'Принимается в обеспеч.']]\
.rename(columns={
    'ISIN ЦБ': 'ISIN',
    'Принимается в обеспеч.': 'NCC collateral'
})\
.dropna()\
.groupby('ISIN')\
.last()\
.reset_index()
securInfo = securInfo[securInfo['NCC collateral']=='Да']

/var/folders/g4/mpm4njl93p547kbhxbvqlm480000gn/T/ipykernel_19167/1088472678.py:2: FutureWarning: Passing bytes to 'read_excel' is deprecated and will be removed in a future version. To read from a byte string, wrap it in a `BytesIO` object.
  securInfo = pd.read_excel(securInfoResponse.content, header=2)[['ISIN ЦБ', 'Принимается в обеспеч.']]\


In [23]:
Prices = Prices.merge(securInfo, how='left', left_on='ISIN', right_on='ISIN')
Prices['NCC collateral'] = Prices['NCC collateral'].fillna('Нет')

##### Шаг 1.2. RuData transform

In [26]:
Prices.columns.tolist()

['ISIN',
 'Name',
 'Type',
 'Fintool',
 'Currency',
 'Interest',
 'Principal',
 'rate',
 'Source',
 'CBR discount 6D',
 'Discount NCC',
 'NCC collateral']

In [25]:
["Name", "Type", "Principal", "Currency", "Clean price pct", "Clean price", "Interest", "Price",
        "Source", "ISIN in DB", "FX Rate", "RUB Price", "CBR discount 6D", "Discount NCC", "NCC collateral"]

['Name',
 'Type',
 'Principal',
 'Currency',
 'Clean price pct',
 'Clean price',
 'Interest',
 'Price',
 'Source',
 'ISIN in DB',
 'FX Rate',
 'RUB Price',
 'CBR discount 6D',
 'Discount NCC',
 'NCC collateral']

In [35]:
RUPriceHistoryLast = RUPriceHistoryDF.groupby('isincode').last().rename(columns={
    'vp_pct': 'Clean price pct',
    'vp_pc': 'Clean price',
    'acc_int': 'InterestHistory'
}).reset_index()
RUPriceHistoryLast[['isincode', 'Clean price pct', 'Clean price', 'InterestHistory']]

,isincode,Clean price pct,Clean price,InterestHistory
0,BYM000001818,98.444900,984.44900,19.428082
1,BYM000001917,98.002824,980.02824,5.955479
2,BYM000001925,97.075172,970.75172,6.465370
3,BYM000001941,96.814731,968.14731,5.605479
4,BYM000002154,99.727585,997.27585,20.054795
...,...,...,...,...
2051,XS2396900685,91.378274,913.78274,14.986111
2052,XS2400040973,91.581019,915.81019,14.812500
2053,XS2401571448,95.760088,957.60088,11.977778
2054,XS2401571521,81.562691,815.62691,15.400000


In [27]:
RUPriceHistoryLast = RUPriceHistoryDF.groupby('isincode').last().rename(columns={
    'vp_pct': 'Clean price pct', 
    'vp_pc': 'Clean price',
    'acc_int': 'InterestHistory'
}).reset_index()
Prices = Prices.merge(
    RUPriceHistoryLast[['isincode', 'Clean price pct', 'Clean price', 'InterestHistory']], how='left', left_on="ISIN", right_on="isincode"
).drop(columns=["isincode"])
Prices["Interest"] = Prices["Interest"].combine_first(Prices["InterestHistory"])
Prices["Interest"].fillna(0, inplace=True)
Prices.drop(columns="InterestHistory", inplace=True)

In [28]:
ccnrd_bonds_idx = Prices["Type"]=='Облигация'
Prices.loc[ccnrd_bonds_idx, "Price"] = Prices.loc[ccnrd_bonds_idx, "Clean price"] + Prices.loc[ccnrd_bonds_idx, "Interest"]
Prices.loc[~ccnrd_bonds_idx, "Price"] = Prices.loc[~ccnrd_bonds_idx, "Clean price"]
Prices.loc[~Prices["Price"].isna(), "Source"] = 'RU Data (ЦЦ НРД)'

In [29]:
ResultPrices = Prices[~Prices["Price"].isna()]
Prices = Prices[Prices["Price"].isna()]

In [33]:
exch_prices = EndOfDayDF.merge(
    ExchangeTree[['id', 'fullname_rus']], how='left', left_on='id_trade_site', right_on='id')[['fintoolId', 'last', 'fullname_rus']]
exch_prices["Source"] = exch_prices['fullname_rus'].apply(lambda x: 'RU Data (' + str(x) + ')')

In [34]:
exch_prices

,fintoolId,last,fullname_rus,Source
0,4200,54.3400,МБ Основной Т+,RU Data (МБ Основной Т+)
1,4215,9.0900,МБ Основной Т+,RU Data (МБ Основной Т+)
2,4217,1596.0000,МБ Основной Т+,RU Data (МБ Основной Т+)
3,4223,82.8000,МБ Основной Т+,RU Data (МБ Основной Т+)
4,4226,943.2000,МБ Основной Т+,RU Data (МБ Основной Т+)
...,...,...,...,...
5051,253300,100.0000,МБ Основной Т+,RU Data (МБ Основной Т+)
5052,253357,101.3900,МБ Основной Т+,RU Data (МБ Основной Т+)
5053,253451,99.9900,МБ Основной Т+,RU Data (МБ Основной Т+)
5054,253485,99.1621,МБ Основной Т+,RU Data (МБ Основной Т+)


In [31]:
Prices = Prices.merge(
    exch_prices, how='left', left_on='Fintool', right_on='fintoolId')

In [123]:
tmp["Source"] = tmp["Source"].combine_first(tmp["fullname_rus"])

In [124]:
tmp["Source"].unique()

array([nan, 'RU Data (ПАО "СПБ" - иностр.)', 'RU Data (ЦЦ НРД)',
       'RU Data (МБ Основной Т+)',
       'RU Data (МБ Основной Т+ (расч. в CNY))', 'RU Data (МБ ОТС)',
       'RU Data (Ценовой центр)',
       'RU Data (RUDIP - оценка облигаций, зарубежный сегмент)'],
      dtype=object)

In [32]:
endofday = ~Prices['fintoolId'].isna()

In [143]:
Prices.loc[endofday, 'Source'] = Prices.loc[endofday, 'fullname_rus']

In [ ]:
Prices.loc[endofday, "Clean price pct"] = Prices.loc[endofday, "PriceEX"]
Prices.loc[endofday, "Clean price"] = exch_bonds["Clean price pct"]/100 * exch_bonds["Principal"]
Prices.loc[endofday, "Price"] = exch_bonds["Clean price"] + exch_bonds["Interest"]

In [99]:
exch_bonds = exch_prices['Type']=='Облигация'

In [ ]:
Prices["Source"] = Prices["Source"].combine_first(Prices["fullname_rus"])

In [ ]:
tmp["Source"] = tmp["Source"].combine_first(tmp["fullname_rus"])
exch_bonds["Clean price pct"] = exch_bonds["PriceEX"]
exch_bonds["Clean price"] = exch_bonds["Clean price pct"]/100 * exch_bonds["Principal"]
exch_bonds["Price"] = exch_bonds["Clean price"] + exch_bonds["Interest"]

In [ ]:
exch_bonds["Clean price pct"] = exch_bonds["PriceEX"]
exch_bonds["Clean price"] = exch_bonds["Clean price pct"]/100 * exch_bonds["Principal"]
exch_bonds["Price"] = exch_bonds["Clean price"] + exch_bonds["Interest"]

In [ ]:
exch_not_bonds = exch_prices[exch_prices['Type']!='Облигация']
exch_not_bonds['Clean price'] = exch_not_bonds['PriceEX']
exch_not_bonds['Price'] = exch_not_bonds['PriceEX']

In [32]:
exch_prices = EndOfDayDF.merge(
    ExchangeTree[['id', 'fullname_rus']], how='left', left_on='id_trade_site', right_on='id')[['fintoolId', 'last', 'fullname_rus']]
exch_prices["Source"] = exch_prices['fullname_rus'].apply(lambda x: 'RU Data (' + str(x) + ')')
exch_prices = exch_prices\
.rename(columns={
    'fintoolId': 'Fintool',
    'last': 'PriceEX'
})\
.drop(columns="fullname_rus")
exch_prices = exch_prices.merge(TransformFintoolReferenceData[["Fintool", "ISIN", "Type", "Principal", "Interest"]], how='inner')

In [33]:
exch_bonds = exch_prices[exch_prices['Type']=='Облигация']
exch_bonds["Clean price pct"] = exch_bonds["PriceEX"]
exch_bonds["Clean price"] = exch_bonds["Clean price pct"]/100 * exch_bonds["Principal"]
exch_bonds["Price"] = exch_bonds["Clean price"] + exch_bonds["Interest"]

/var/folders/g4/mpm4njl93p547kbhxbvqlm480000gn/T/ipykernel_2901/1057747574.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exch_bonds["Clean price pct"] = exch_bonds["PriceEX"]
/var/folders/g4/mpm4njl93p547kbhxbvqlm480000gn/T/ipykernel_2901/1057747574.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exch_bonds["Clean price"] = exch_bonds["Clean price pct"]/100 * exch_bonds["Principal"]
/var/folders/g4/mpm4njl93p547kbhxbvqlm480000gn/T/ipykernel_2901/1057747574.py:4: SettingWithCopyWarning: 
A value is

In [34]:
exch_not_bonds = exch_prices[exch_prices['Type']!='Облигация']
exch_not_bonds['Clean price'] = exch_not_bonds['PriceEX']
exch_not_bonds['Price'] = exch_not_bonds['PriceEX']

/var/folders/g4/mpm4njl93p547kbhxbvqlm480000gn/T/ipykernel_2901/1797406440.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exch_not_bonds['Clean price'] = exch_not_bonds['PriceEX']
/var/folders/g4/mpm4njl93p547kbhxbvqlm480000gn/T/ipykernel_2901/1797406440.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exch_not_bonds['Price'] = exch_not_bonds['PriceEX']


In [35]:
Prices.update(exch_bonds.drop_duplicates().set_index("ISIN"))
Prices.update(exch_not_bonds.drop_duplicates().set_index("ISIN"))

In [36]:
ResultPrices = pd.concat([ResultPrices, Prices[~Prices['Price'].isna()]], axis=0)
Prices = Prices[Prices['Price'].isna()]

##### Шаг 3.1. Tranform

In [54]:
TransformHistoryStockBonds = HistoryStockBondsDF.groupby("isin")['close'].last().rename("Price").rename_axis("ISIN")
TransformHistoryStockShares = HistoryStockSharesDF.groupby("isin")['close'].last().rename("Price").rename_axis("ISIN")
TransformHistoryStockNdm = HistoryStockNdmDF.groupby("isin")['close'].last().rename("Price").rename_axis("ISIN")
TransformHistoryStockCcp = HistoryStockCcpDF.groupby("isin")['close'].last().rename("Price").rename_axis("ISIN")

In [55]:
Prices.update(TransformHistoryStockBonds)
Prices.update(TransformHistoryStockShares)
Prices.update(TransformHistoryStockNdm)
Prices.update(TransformHistoryStockCcp)

In [56]:
moex_idx = Prices[~Prices["Price"].isna()].index
Prices.loc[moex_idx, 'Source'] = 'RU Data (MOEX)'
# Prices.loc[moex_idx, 'ISIN in DB'] = 1

In [57]:
ResultPrices = pd.concat([ResultPrices, Prices.loc[moex_idx]], axis=0)
Prices = Prices[Prices['Price'].isna()]

#### Шаг 4. Finish

In [58]:
FullPrices = pd.concat([ResultPrices, Prices], axis=0)

##### Шаг 4.1. Update currency

In [59]:
FullPrices = FullPrices\
.reset_index()\
.merge(CurrencyRateDF[['currency', 'rate']], how='left', left_on='Currency', right_on='currency')\
.set_index("ISIN")

In [60]:
rub_price_idx = FullPrices[FullPrices["RUB Price"].isna()].index
FullPrices.loc[rub_price_idx, "FX Rate"] = FullPrices.loc[rub_price_idx, "rate"].fillna(1)
FullPrices.loc[rub_price_idx, "RUB Price"] = FullPrices.loc[rub_price_idx, "Price"] * FullPrices.loc[rub_price_idx, "FX Rate"]

In [61]:
FullPrices.drop(columns=['currency', 'rate'], inplace=True)

##### Шаг 4.2. Write excel

In [62]:
FullPrices['export_timestamp'] = dt.now().replace(microsecond=0)
FullPrices['Date'] = last_day_month
FullPrices['Source'] = FullPrices['Source'].fillna(0)
FullPrices['ISIN in DB'] = FullPrices['ISIN in DB'].fillna(0)
ResultPrices = FullPrices[~FullPrices['Price'].isna()]

In [63]:
with pd.ExcelWriter(f'../data/Output/prices.xlsx', datetime_format='dd.mm.yyyy') as writer:
    ResultPrices.to_excel(writer)

In [67]:
ResultPrices.to_csv('./data/Output/prices.csv', index=False)

In [68]:
from src.utils.send_email import main
main("prices.xlsx", "prices.csv")